In [1]:
from testproblems import generate_random_prob
from inputresolver import getallvars, direct_eqs, invert_edges
from datastructures.polycasebuilder import generate_random_polynomials, get_arg_mapping, directed_poly_executables, eqv_to_edges_tree

In [2]:
from datastructures.graphutils import all_variables, all_edges, edges_to_Ein_Eout, flat_graph_formulation
from datastructures.tearing import dir_graph

In [3]:
from datastructures.transformations import partial_inversion
from datastructures.execution import Component

In [4]:
import numpy as np
import networkx as nx
import sympy as sp

In [5]:
from compute import Var

In [6]:
x,y,a = Var('x'), Var('y'), Var('a')

### Test that we end up with the same expression

In [7]:
old_output = a
old_expression = x/y+1
new_output = None

In [8]:
new_expression = partial_inversion(old_expression, old_output, new_output)

In [9]:
new_expression

a*y - x - y

In [152]:
partial_inversion(new_expression, None, a)

x/y + 1

### Test on small example

In [13]:
#(5,10,7,1.7 no numerical issues)
n_eqs = 2
n_vars = 3
seed = 9#8 is triang#seed 10 is nice 42 
sparsity = 1.7#0.8 1.1 #1.7 1.3
eqv, varinc, output_set = generate_random_prob(n_eqs, n_vars, seed, sparsity)
rng = np.random.default_rng(seed)
polynomials, var_mapping, edges, tree, components = generate_random_polynomials(eqv, output_set, n_eqs, rng)

In [14]:
components = [comp for comp in components if comp.outputs != (None,)]

In [15]:
edges

({0: (0, 2), 1: (1, 2)}, {0: (1,), 1: (0,)}, {})

In [56]:
def var_from_mapping(arg_mapping, Eout, comp):
    if Eout[comp][0] is not None:
        return arg_mapping[Eout[comp][0]]
    else:
        return None

In [75]:
nedges = ({
    0: (0,1),
    1: (1,2),
},{
    0: (2,),
    1: (None,)
}, {1: (0,)})

In [70]:
oldedges = edges
newedges = nedges
arg_mapping = get_arg_mapping(var_mapping, True)

In [76]:
def transform_components(oldedges, newedges, components, arg_mapping):
    Ein, Eout = edges_to_Ein_Eout(oldedges)
    old_edges = flat_graph_formulation(*oldedges).to_undirected()
    new_edges = flat_graph_formulation(*newedges).to_undirected()
    assert nx.is_isomorphic(old_edges, new_edges)
    newEin, newEout = edges_to_Ein_Eout(nedges)
    new_components = []
    for idx,comp in enumerate(components):
        compid = comp.component
        newcompid = len(components)+idx
        old_out = var_from_mapping(arg_mapping, Eout, compid)
        new_out = var_from_mapping(arg_mapping, newEout, compid) 
        new_function_expression = partial_inversion(comp.fxdisp, old_out, new_out)
        newcomponent = Component.fromsympy(new_function_expression, new_out, component=newcompid, arg_mapping=arg_mapping)
        new_components.append(newcomponent)
    return new_components

In [74]:
components

[((0, 2), 0, (1,), '-3.0/(x_0*x_2)'), ((2, 1), 1, (0,), '-0.875/(x_1*x_2)')]

In [71]:
transform_components(edges, nedges, components, arg_mapping)

[((0, 1), 2, (2,), '-3.0/(x_0*x_1)'),
 ((0, 2, 1), 3, (None,), 'x_0*x_1*x_2 + 0.875')]